In [2]:
%load_ext autoreload
%autoreload 2 

In [3]:
import warnings
warnings.filterwarnings("ignore")
import sys  
sys.path.insert(1, '/home/manying/Projects/fcgFewShot')

In [3]:
from utils import load_config
options = load_config("../config/config_label_prop_openset.json")

In [4]:
from loadDataset import LoadDataset
from fcgVectorize import FCGVectorize
from trainModule import TrainModule
from trainModule import TestModule
import os

dataset = LoadDataset(options, pretrain=False)

Loading all datasets...
train dataset shape: (1180, 16)
train dataset family number: 59
test dataset shape: (200, 16)
test dataset family number: 10
val dataset shape: (200, 16)
val dataset family number: 10
Loading openset data...
Openset data shape: (171, 16)


In [5]:
vectorizer = FCGVectorize(options, dataset)
vectorizer.node_embedding(dataset.opensetData, openset=True)

Word2vec model exist, load word2vec model...
Start to get node embedding...


100%|██████████| 171/171 [00:00<00:00, 25147.29it/s]

Finish getting node embedding


In [6]:
trainModule = TrainModule(options, dataset)

Setting up the training module...
Loading data from ../embeddings/x86_64_withVal_withPretrain_ghidra_888/word2vec...
Loading training data...
Loading data from ../embeddings/x86_64_withVal_withPretrain_ghidra_888/word2vec/trainData.pkl...
Loading validation data...
Loading data from ../embeddings/x86_64_withVal_withPretrain_ghidra_888/word2vec/valData.pkl...
Device: cuda:0
Model: LabelPropagation(
  (encoder): GraphSAGELayer(
    (sage_convs): ModuleList(
      (0-2): 3 x SAGEConv(128, 128, aggr=mean)
    )
    (norms): ModuleList(
      (0-2): 3 x BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (relation): GraphRelationNetwork(
    (sage): GraphSAGELayer(
      (sage_convs): ModuleList(
        (0): SAGEConv(128, 64, aggr=mean)
        (1): SAGEConv(64, 32, aggr=mean)
      )
      (norms): ModuleList(
        (0): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): BatchNorm1d(32, eps=1e-05, momentu

In [ ]:
import torch
testConfigPath = "/home/manying/Projects/fcgFewShot/checkpoints/x86_64_withVal_withPretrain_ghidra_888_baseline/10way_5shot_LabelPropagation_alpha0.7_k20_20250316_133650/config.json"

testModule = TestModule(testConfigPath, dataset, options)

testLoader = testModule.testLoader
opensetLoader = testModule.opensetLoader

accuracy_query = []
accuracy_openset = []

testModule.load_best_model()
for testData in testLoader:
    testModule.model.eval()
    testData = testData.to(testModule.device)
    with torch.no_grad():
        testDataEncoded = testModule.model.get_encoded_data(testData)
        avg_dist = testModule.model.get_avg_distance_between_support(testDataEncoded, testData.y)
        print("Average distance: ", avg_dist)
        for opensetData in opensetLoader:
            predicts = []
            opensetData = opensetData.to(testModule.device)
            testModule.model.eval()
            with torch.no_grad():
                testDataEncoded = testModule.model.get_encoded_data(testData)
                opensetDataEncoded = testModule.model.get_encoded_data(opensetData)
                dists = testModule.model.get_openset_distance_between_support(testDataEncoded, testData.y, opensetData)  
                for i in range(dists.shape[0]):
                    min_dist = torch.min(dists[i])
                    if min_dist <= avg_dist:
                        predicts.append(0) # not openset
                    else:
                        predicts.append(1) # openset
                print(dists.shape)
            # count the number of openset samples
            num_openset = sum(predicts)
            print("Accuracy: ", (num_openset) / len(predicts))
            accuracy_openset.append((num_openset) / len(predicts))
            break
        testDataEncoded = testModule.model.get_encoded_data(testData)
        query_dist = testModule.model.get_query_distance_between_support(testDataEncoded, testData.y)
        predicts = []
        for i in range(query_dist.shape[0]):
            min_dist = torch.min(query_dist[i])
            if min_dist >= avg_dist:
                predicts.append(0) # not query
            else:   
                predicts.append(1)

    break


    

Setting up the testing module...
Loading data from ../embeddings/x86_64_withVal_withPretrain_ghidra_888/word2vec...
Loading testing data...
Loading data from ../embeddings/x86_64_withVal_withPretrain_ghidra_888/word2vec/testData.pkl...
Loading openset data...
Loading openset data...
Loading data from ../embeddings/x86_64_withVal_withPretrain_ghidra_888/word2vec/opensetData_random_0.1.pkl...
Finish setting up the testing module
Model loaded from /home/manying/Projects/fcgFewShot/checkpoints/x86_64_withVal_withPretrain_ghidra_888_baseline/10way_5shot_LabelPropagation_alpha0.7_k20_20250316_133650/epoch_42_0.8847333538532257_best.pth
Class 0: Average distance = 73166.0078125
Class 1: Average distance = 80139.015625
Class 2: Average distance = 16068.0537109375
Class 3: Average distance = 74.07701873779297
Class 4: Average distance = 102308.5
Class 5: Average distance = 28627.884765625
Class 6: Average distance = 12941.2236328125
Class 7: Average distance = 1152841344.0
Class 8: Average dist

### Nn model

In [4]:
from utils import load_config
options = load_config("../config/config_nn_openset.json")

In [5]:
from loadDataset import LoadDataset
from fcgVectorize import FCGVectorize
from trainModule import TrainModule
from trainModule import TestModule
import os

dataset = LoadDataset(options, pretrain=False)

Loading all datasets...
train dataset shape: (1180, 16)
train dataset family number: 59
test dataset shape: (200, 16)
test dataset family number: 10
val dataset shape: (200, 16)
val dataset family number: 10
Loading openset data...
Openset data shape: (171, 16)


In [6]:
trainModule = TrainModule(options, dataset)

Setting up the training module...
Loading data from ../embeddings/x86_64_withVal_withPretrain_ghidra_888/word2vec...
Loading training data...
Loading data from ../embeddings/x86_64_withVal_withPretrain_ghidra_888/word2vec/trainData.pkl...
Loading validation data...
Loading data from ../embeddings/x86_64_withVal_withPretrain_ghidra_888/word2vec/valData.pkl...
Device: cuda:0
Model: GraphSAGE(
  (sage_convs): ModuleList(
    (0-1): 2 x SAGEConv(128, 128, aggr=mean)
  )
  (norms): ModuleList(
    (0-1): 2 x BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (output_proj): Sequential(
    (0): Linear(in_features=128, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
)
Loss function: <loss.NnLoss object at 0x7f559e819f10>
Optimizer: AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    fo

In [7]:
import torch
import pickle
testConfigPath = "/home/manying/Projects/fcgFewShot/checkpoints/x86_64_withVal_withPretrain_ghidra_888_baseline/10way_5shot_NnNet_with_pretrain_20250108_221055/config.json"

testModule = TestModule(testConfigPath, dataset, options)

testLoader = testModule.testLoader
opensetLoader = testModule.opensetLoader

accuracy_query = []
accuracy_openset = []

testModule.load_best_model()
for testData in testLoader:
    testModule.model.eval()
    testData = testData.to(testModule.device)
    with torch.no_grad():
        testDataEncoded = testModule.model(testData)
        avg_dist = testModule.loss_fn.get_avg_distance_between_support(testDataEncoded, testData.y)
        print("Average distance: ", avg_dist)
        for opensetData in opensetLoader:
            predicts = []
            opensetData = opensetData.to(testModule.device)
            testModule.model.eval()
            with torch.no_grad():
                testDataEncoded = testModule.model(testData)
                opensetDataEncoded = testModule.model(opensetData)
                dists = testModule.loss_fn.get_openset_distance_between_support(testDataEncoded, testData.y, opensetDataEncoded)  
                labelDictOpenset = {}
                with open("../embeddings/x86_64_withVal_withPretrain_ghidra_888/word2vec/labelDict_openset_random_0.1.pkl", "rb") as f:
                    labelDictOpenset = pickle.load(f)
                print("labelDictOpenset: ", labelDictOpenset)
                for i in range(dists.shape[0]):
                    min_dist = torch.min(dists[i])
                    print("min_dist: ", min_dist, "Label: ", opensetData.y[i])
                    if min_dist <= avg_dist:
                        predicts.append(0) # not openset
                    else:
                        predicts.append(1) # openset
            # count the number of openset samples
            num_openset = sum(predicts)
            print("Accuracy: ", (num_openset) / len(predicts))
            accuracy_openset.append((num_openset) / len(predicts))
            break
        testDataEncoded = testModule.model(testData)
        query_dist = testModule.loss_fn.get_query_distance_between_support(testDataEncoded, testData.y)
        predicts = []
        for i in range(query_dist.shape[0]):
            min_dist = torch.min(query_dist[i])
            if min_dist >= avg_dist:
                predicts.append(0) # not query
            else:   
                predicts.append(1)

    break

Setting up the testing module...
Loading data from ../embeddings/x86_64_withVal_withPretrain_ghidra_888/word2vec...
Loading testing data...
Loading data from ../embeddings/x86_64_withVal_withPretrain_ghidra_888/word2vec/testData.pkl...
Loading openset data...
Loading openset data...
Loading data from ../embeddings/x86_64_withVal_withPretrain_ghidra_888/word2vec/opensetData_random_0.1.pkl...
Finish setting up the testing module
Model loaded from /home/manying/Projects/fcgFewShot/checkpoints/x86_64_withVal_withPretrain_ghidra_888_baseline/10way_5shot_NnNet_with_pretrain_20250108_221055/epoch_25_best.pth
Class 0: Average distance = 28.6367130279541
Class 1: Average distance = 4.132623195648193
Class 2: Average distance = 0.6481151580810547
Class 3: Average distance = 6.461498088021706e-15
Class 4: Average distance = 2.3675599098205566
Class 5: Average distance = 2.337858200073242
Class 6: Average distance = 0.30937156081199646
Class 7: Average distance = 51.00477981567383
Class 8: Average